In [2]:
import os
import re
import string
import glob
import nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from nltk.corpus import stopwords
from collections import Counter
from nltk import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [12]:
text_dir = "../data/raw/text"
labels_dir =  "../data/raw/labels"
audio_dir = "../data/raw/audio"



In [33]:
text_files = glob.glob(f"{text_dir}/*.txt")


def remove_stamps_str(line)->str:
    #clip_num = re.search('.+___\d\d?\d?___.+', line).group(0)
    stamp = re.search('.+___', line).group(0)
    new_line = line.strip(stamp)
    return new_line

In [44]:
test_str = "n-sgVVTE9Io___3___25.613___29.038___So it is very critical to set a budget."
taco = re.search
re.sub('___\d\d?\d?___', '\d', test_str).group(0)

error: bad escape \d at position 0

In [22]:
#Retire chaque stamps, chaque texte devien 1 seul str
corpus = []
text = []
for filename in text_files:
    text_str = ''
    with open(file=filename, encoding='utf-8') as f:
        lines = f.readlines()
        for count, line in enumerate(lines):
            clean_line = remove_stamps_str(line)
            text.append(clean_line)
    text_list.append(text)

In [55]:
stop_words = stopwords.words('English')

#création d'un dict pour lookup en O(1)
stopwords_dict = Counter(stop_words)


def avg_count(l1:list):
    word_count = 0
    for e in l1:
        word_count += len(e.split())
    return int(word_count/len(l1))

#Retire tous les timestamps en début de ligne, présents dans chaque transcript
def remove_stamps_str(line)->str:
    stamp = re.search('.+___', line).group(0)
    new_line = line.strip(stamp)
    return new_line

#Retire les charactères non-ascii 
def remove_nonascii(line)->str:
    ascii_line = line.encode(encoding = 'ascii', errors = 'ignore').decode()
    return ascii_line

#met tout en minuscules, retire les nombres et stopwords
def clean_stopwords_digits(line)->str:
    new_line = ' '.join([word.lower() for word in line.split() if (len(word) >=2 and word.isalpha() and word not in stopwords_dict)])
    return new_line



In [8]:
id_label_df = pd.read_csv("../data/interim/labels/interim.csv")
display(id_label_df)

,id,anger,disgust,fear,happiness,sadness,surprise,sentiment
0,--qXJuDtHPw_5,0,0,0,1,0,0,1
1,-3g5yACwYnA_10,0,0,0,0,0,0,1
2,-3g5yACwYnA_13,0,0,0,0,0,0,1
3,-3g5yACwYnA_2,0,0,0,1,0,0,0
4,-3g5yACwYnA_3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
21860,zwTrXwi54us_6,0,0,0,0,0,0,0
21861,zwTrXwi54us_7,0,0,0,0,0,0,0
21862,zwTrXwi54us_8,0,0,0,0,0,0,0
21863,zwTrXwi54us_9,0,0,0,0,0,0,1


str

In [ ]:
### word_tokenize utile? Lent, semble pas nécessaire, peut juste split?

In [64]:
df_text = pd.DataFrame((np.array(text_list, dtype= object)).reshape(-1))
df_text.columns = ['raw_text']

df_text['raw_text'] = df_text['raw_text'].apply(lambda x : ' '.join(x))
df_text['pre_cleaned_text'] = df_text['raw_text'].apply(remove_nonascii)

#on utilise la librairie NLTK pour sortir que les "tokens"



df_text['tokens'] = df_text['pre_cleaned_text'].apply(lambda l : word_tokenize(''.join(clean_stopwords_digits(l))))


df_text['lines'] = df_text['raw_text'].apply(lambda x : len(x))
df_text['avg_count'] = df_text['raw_text'].apply(avg_count)
df_text['word_count'] = df_text['tokens'].apply(len)



display(df_text)

,raw_text,pre_cleaned_text,tokens,lines,avg_count,word_count
0,I see that there are three category of writers...,I see that there are three category of writers...,"[see, three, category, define, story, an, like...",771,0,49
1,"Hi, my name is Raj Shah. I'm from Spectro Coat...","Hi, my name is Raj Shah. I'm from Spectro Coat...","[name, raj, spectro, vice, president, key, pol...",2044,0,144
2,"Introduction Speaker: Dr. Erma Jean Sims, Sono...","Introduction Speaker: Dr. Erma Jean Sims, Sono...","[introduction, erma, jean, sonoma, state, univ...",650,0,51
3,"This is Rhett Reiger, White Caspian Studios on...","This is Rhett Reiger, White Caspian Studios on...","[this, rhett, white, caspian, studios, behalf,...",1501,0,110
4,This is financial adviser Patrick Munro answer...,This is financial adviser Patrick Munro answer...,"[this, financial, adviser, patrick, munro, ans...",1055,0,80
...,...,...,...,...,...,...
3832,Hi Guys I just wanted to share my thoughts wit...,Hi Guys I just wanted to share my thoughts wit...,"[hi, guys, wanted, share, thoughts, book, resu...",1117,0,103
3833,- University of Illinois candidates are very p...,- University of Illinois candidates are very p...,"[university, illinois, candidates, prepared, g...",759,0,43
3834,"e use both, again, we use Aprimo software inte...","e use both, again, we use Aprimo software inte...","[use, use, aprimo, software, internally, use, ...",595,0,39
3835,"Well, I think for us it’s-- the way to leverag...","Well, I think for us its-- the way to leverage...","[think, us, way, leverage, technology, innovat...",1957,0,150


In [10]:
nltk.download('averaged_perceptron_tagger')
df_text['tagged'] = df_text['tokens'].apply(nltk.tag.pos_tag )

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\trott\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

Bag of words

In [ ]:
import gensim.downloader as api

w2v_model = api.load("word2vec-google-news-300")

#SEULEMENT PRENDRE MOT DANS MODEL.VOCAB